In [1]:
from lib.model import fromcrn
from get_bifurcations import get_bifurcations
from scipy.interpolate import UnivariateSpline
from numpy import *

from lib.colors import cyan,yellow
from matplotlib.pyplot import *
from numpy import *
from scipy.interpolate import interp2d
from json import loads,dumps

from os import system
from os.path import isfile
from re import finditer,sub,search

from re import sub,search,finditer,findall
from yaml import load

In [2]:
file_path = 'double-exclusive.crn'
model = fromcrn(file_path)

In [28]:
from sympy.functions.combinatorial.factorials import binomial
from scipy.special import comb
pattern = r'(?<=->)[ ]*[\[\{].*[\]\}]'
for reaction in model.reactions :

    match = search(pattern,reaction)
    rate = match.group()
    
    reaction = reaction.replace(rate,'')

    rate = rate.replace('^','**')
    rate = rate.replace('{','').replace('}','')
    rate = rate.replace('[','').replace(']','')

    reactants,products = reaction.split('->')
    print rate,reactants

self.growth*self.luxR self.luxR 
self.growth*self.lasR self.lasR 
self.growth*self.laci self.laci 
self.growth*self.tetr self.tetr 
self.capacity*self.aR33/(1+self.tetr**self.nT) 
 self.dR  self.luxR 
self.capacity*self.aS175/(1+self.laci**self.nL) 
self.dS  self.lasR 
self.capacity*self.aL*self.P76 
self.dL  self.laci 
self.capacity*self.aT*self.P81 
self.dT  self.tetr 


array([0., 0., 4.])

In [36]:
prod([ comb(model.states[key],3) for key in model.states ],axis=0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
def calculate(model,c6,c12,L,T):
    
    Lattractor = array([
        L[argmin(abs(c12[:,0]-c12x)),argmin(abs(c6[0]-c6x))]
        for c6x,c12x in model.state['diffusables'] ]).reshape(-1)

    Tattractor = array([
        T[argmin(abs(c12[:,0]-c12x)),argmin(abs(c6[0]-c6x))]
        for c6x,c12x in model.state['diffusables'] ]).reshape(-1)

    space = linspace(0,model.xmax,len(Lattractor))
    return space,Lattractor,Tattractor

def generate_frame(j,model,space,L,T):
    '''main program figure display'''

    figure(figsize=(10,10))

    # plot scaffolds
    plot(space,10**L,'.',color='darkcyan')
    plot(space,10**T,'.',color='gold')
    plot(-1,-1,'k.',label='Local Steady State')

    # plot system state
    plot(model.space,model.state['inhibitors'].T[0],color='darkcyan')
    plot(model.space,model.state['inhibitors'].T[1],color='gold')
    plot(-1,-1,'k',label='Concentration at $t = {}h$'.format(model.time[-1]))

    legend(fontsize=16)
    xlim(0,model.xmax)
    
    ylabel(r'Inhibitors $L(x,t),T(x,t)$ / nM',fontsize=16);
    xlabel(r'Space $x$ / cm',fontsize=16);
    yscale('log')

    savefig(str(j).zfill(4)+'.png')
    close()

    return j+1
    
def create_animation(C6,C12):
    '''execute system commands to create animation from frames'''
    system('convert -delay 10 -loop 0 *.png c6-{}_c12-{}.gif'.format(C6,C12))
    system('rm *.png')

In [ ]:
C6=120;C12=40
t_final = 750.0
p = 0.01

# import model from file
model = fromcrn(crn_path)
model.dt = 0.25

# initial condition
model.state['diffusables'][:,0][model.space<p] = model.xmax * C6 / p
model.state['diffusables'][:,1][model.space>(model.xmax-p)] = model.xmax * C12 / p

# create bifrucation figure
i,j = 0,0
while model.time[-1] < t_final :

    model.time_step()
    space,Lattractor,Tattractor = calculate(model,c6,c12,L,T)
    
    if i % 10 == 0 :
        j = generate_frame(j,model,space,Lattractor,Tattractor)
    i += 1

create_animation(C6,C12)

In [ ]:
model.time[-1]